In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
list_file = 'experiments_list.txt'
with open(list_file, 'r') as f:
    dirs = [f'experiments/{line.strip()}' for line in f if line.strip() and not line.strip().startswith('#')]

In [3]:
# Load logs.csv from each directory into a dictionary of DataFrames
dfs = {}
for d in dirs:
    csv_path = os.path.join(d, 'logs.csv')
    if os.path.isfile(csv_path):
        dfs[d] = pd.read_csv(csv_path)
    else:
        print(f'Warning: {csv_path} not found, skipping.')

# Optionally, concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(
    dfs.values(), 
    keys=dfs.keys(), 
    names=['experiment', 'row']
).reset_index(level='row', drop=True).reset_index().rename(columns={'index': 'experiment'})

# Display the combined DataFrame
df = combined_df.query('itr>0')[['experiment','iteration_time']].copy().groupby('experiment').mean().reset_index()

In [4]:
parts = df.experiment.str.split('/', expand=True)
df['Experiment'] = parts[1]
df['Method'] = parts[2]
df['Loss'] = parts[3]

In [ ]:
df['Loss'].replace(
    {'SOCM_adjoint': 'SOCM (adjoint)',
     'log_variance': 'Log variance',
     'rel_entropy': 'Relative entropy',
     'pinn_GELU': 'PINN',
     'rel_GELU': 'Relative loss',
     'ritz_GELU': 'Deep ritz loss',
     'var_GELU': 'Variational loss'},
     inplace=True
)

df['Experiment'].replace(
    {'OU_anisotropic_d20': 'AQ',
     'OU_stable_d20': 'IQ',
     'OU_hard_d20': 'RQ',
     'double_well_d10': 'DW'},
     inplace=True
)

/home/lclaeys/tmp/ipykernel_3202678/4078013571.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Loss'].replace(
/home/lclaeys/tmp/ipykernel_3202678/4078013571.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [6]:
pivot = df.pivot_table(
    index=['Method', 'Loss'],
    columns='Experiment',
    values='iteration_time',
    aggfunc='mean'
)

In [7]:
display(pivot)

Experiment                       AQ        DW        IQ        RQ
Method   Loss                                                    
COMBINED Log variance      0.331186  0.744688  0.328335  0.328471
         Relative entropy  0.419320  0.390957  0.413951  0.418798
         SOCM              0.431479  0.398070  0.432200  0.432274
         adjoint_matching  0.227224  0.312696  0.569376  0.331509
EIGF     pinn_GELU         0.587298  1.517855  0.583295  0.661866
         rel_GELU          0.586972  1.539109  0.583593  0.661570
         ritz_GELU         0.222887  1.309628  0.226832  0.227275
         var_GELU          0.072800  1.312096  0.072405  0.228154
FBSDE    FBSDE             0.453729  0.451868  0.445013  0.443269
IDO      Log variance      0.232888  0.192729  0.207384  0.212383
         Relative entropy  7.637422  0.374573  0.398909  0.413282
         SOCM              1.152043  0.507702  0.807243  0.798727
         adjoint_matching  0.180337  0.209139  0.281918  0.229828

In [8]:
latex_str = pivot.to_latex(
    caption='Iteration times by experiment, method and loss',
    label='tab:iteration_times',
    escape=True,            # <-- ensure special chars (_ & %) get backslash-escaped
    multicolumn=True,
    multicolumn_format='c',
    float_format="%.3f",
    bold_rows=True
)
print(latex_str)

\begin{table}
\caption{Iteration times by experiment, method and loss}
\label{tab:iteration_times}
\begin{tabular}{llrrrr}
\toprule
 & Experiment & AQ & DW & IQ & RQ \\
Method & Loss &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{\textbf{COMBINED}} & \textbf{Log variance} & 0.331 & 0.745 & 0.328 & 0.328 \\
\textbf{} & \textbf{Relative entropy} & 0.419 & 0.391 & 0.414 & 0.419 \\
\textbf{} & \textbf{SOCM} & 0.431 & 0.398 & 0.432 & 0.432 \\
\textbf{} & \textbf{adjoint\_matching} & 0.227 & 0.313 & 0.569 & 0.332 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{EIGF}} & \textbf{pinn\_GELU} & 0.587 & 1.518 & 0.583 & 0.662 \\
\textbf{} & \textbf{rel\_GELU} & 0.587 & 1.539 & 0.584 & 0.662 \\
\textbf{} & \textbf{ritz\_GELU} & 0.223 & 1.310 & 0.227 & 0.227 \\
\textbf{} & \textbf{var\_GELU} & 0.073 & 1.312 & 0.072 & 0.228 \\
\cline{1-6}
\textbf{FBSDE} & \textbf{FBSDE} & 0.454 & 0.452 & 0.445 & 0.443 \\
\cline{1-6}
\multirow[t]{4}{*}{\textbf{IDO}} & \textbf{Log variance} & 0.233 & 0.193 & 0.207 & 0.212 \\
\te